### import the main functions

In [2]:
#!/usr/bin/env python
__author__ = 'heroico'

import logging
import numpy
import os
import gzip
import ntpath
import metax.WeightDBUtilities as WeightDBUtilities
import metax.PrediXcanFormatUtilities as PrediXcanFormatUtilities
import metax.ThousandGenomesUtilities as ThousandGenomesUtilities
import metax.Logging as Logging
import metax.Utilities as Utilities
import metax.Formats as Formats


def pathLeaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

class ProcessWeightDB(object):
    def __init__(self, args):
        self.weight_db = pathLeaf(args.weight_db)
        self.db_path = args.weight_db
        self.data_folder = args.input_folder
        self.correlation_output = args.correlation_output
        self.covariance_output = args.covariance_output
        if args.covariance_output is None:
            comp = os.path.splitext(self.weight_db)[0]
            name = comp + ".cov.txt.gz"
            path = os.path.join("intermediate", "cov")
            path = os.path.join(path, name)
            self.covariance_output = path

        self.input_format = args.input_format

        self.found_genes_for_covariance = {}
        self.found_genes_for_correlation = {}

        self.min_maf_filter = float(args.min_maf_filter) if args.min_maf_filter else None
        self.max_maf_filter = float(args.max_maf_filter) if args.max_maf_filter else None

        self.max_snps_in_gene = int(args.max_snps_in_gene) if args.max_snps_in_gene else None

    def run(self):
        if not self.correlation_output and not self.covariance_output:
            logging.info("Provide --correlation_output or --covariance_output or both")
            return

        logging.info("Loading Weights")
        weight_db_logic = WeightDBUtilities.WeightDBEntryLogic(self.db_path)

        logging.info("Building files")
        self.buildFiles(weight_db_logic)

        logging.info("Ran successfully")

    def buildFiles(self, weight_db_logic):
        do_correlations = self.correlation_output is not None
        if do_correlations:
            if os.path.exists(self.correlation_output):
                logging.info("%s already exists, delete it if you want it figured out again", self.correlation_output)
                do_correlations = False
            else:
                correlation_dir = os.path.dirname(self.correlation_output)
                if not os.path.exists(correlation_dir):
                    os.makedirs(correlation_dir)
                self.writeFileHeader(self.correlation_output)

        do_covariances = self.covariance_output is not None
        if do_covariances:
            if os.path.exists(self.covariance_output):
                logging.info("%s already exists, delete it if you want it figured out again", self.covariance_output)
                do_covariances = False
            else:
                covariance_dir = os.path.dirname(self.covariance_output)
                if not os.path.exists(covariance_dir):
                    os.makedirs(covariance_dir)
                self.writeFileHeader(self.covariance_output)

        if not do_covariances and not do_correlations:
            return

        names = Utilities.dosageNamesFromFolder(self.data_folder)
        for name in names:
            snps, snps_by_rsid = self.getSNPS(name, weight_db_logic)
            if do_correlations:
                self.addToCorrelationFile(weight_db_logic, name, snps, snps_by_rsid)

            if do_covariances:
                self.addToCovarianceFile(weight_db_logic, name, snps, snps_by_rsid)

    def writeFileHeader(self,path):
        with gzip.open(path, "ab") as file:
            file.write("GENE RSID1 RSID2 VALUE\n")

    def getSNPS(self, name, weight_db_logic):
        dosageLoader = None
        if self.input_format == Formats.IMPUTE:
            dosageLoader = ThousandGenomesUtilities.IMPUTEDosageLoader(self.data_folder, name) #outdated code
        elif self.input_format == Formats.PrediXcan:
            dosageName = Utilities.dosageName(name)
            path = os.path.join(self.data_folder, dosageName)
            dosageLoader = PrediXcanFormatUtilities.PrediXcanFormatDosageLoader(path, weight_db_logic)
        else:
            logging.info("Invalid input format: %s", self.input_format)
            return
        snps, snps_by_rsid = dosageLoader.load()
        return snps, snps_by_rsid

    def addToCovarianceFile(self, weight_db_logic, name, snps, snps_by_rsid):
        logging.info("Adding to covariance for %s-%s", name, self.weight_db)

        genes = weight_db_logic.weights_by_gene.keys()
        total_genes = len(genes)
        last_reported_percent = 0
        processed = 0
        for gene in genes:
            processed += 1
            percent = int(processed*100.0 / total_genes)
            if percent == last_reported_percent+10:
                logging.info("%d percent genes processed", percent)
                last_reported_percent = percent

            entries = self.buildCovarianceEntries(name, gene, weight_db_logic, snps_by_rsid)

            if len(entries) == 0:
                logging.log(6,"Gene %s has no snps in current file", gene)
                continue

            self.addToFile(self.covariance_output, gene, entries)

    def addToFile(self, path, gene, entries):
        with gzip.open(path, "ab") as file:
            for entry in entries:
                line = " ".join([gene, entry[0], entry[1], entry[2]])+"\n"
                file.write(line)

    def buildCovarianceEntries(self, name, gene, weight_db_logic, snps_by_rsid):
        weights_in_gene = weight_db_logic.weights_by_gene[gene]
        rsids_from_genes = weights_in_gene.keys()

        #gather as much data as we can work on
        related_rsids, related_data = self.buildRelatedData(rsids_from_genes, snps_by_rsid, weights_in_gene)

        if len(related_rsids) == 0:
            return []

        self.updateFoundCovariance(gene, name)

        #covariance matrix of related SNP's data
        array = numpy.array(related_data)
        cov = numpy.cov(array)

        #translate into sql entries
        entries = self.buildMatrixOutputEntries(cov, rsids_from_genes, related_rsids, snps_by_rsid)
        if not len(entries):
            raise NameError("Couldn not build covariance entries for (%s,%s)" %(name,gene))
        return entries

    def updateFoundCovariance(self, gene, name):
        found = None
        if gene in self.found_genes_for_covariance:
            found = self.found_genes_for_covariance[gene]
            logging.info("Gene %s found again for %s", gene, name)
        else:
            found = []
            self.found_genes_for_covariance[gene] = found
        found.append(name)

    def buildRelatedData(self, rsids_from_genes, snps_by_rsid, weights_in_gene):
        related_rsids = []
        related_data = []

        l = len(rsids_from_genes)
        if self.max_snps_in_gene and l > self.max_snps_in_gene:
            logging.info("Skipping covariance too large: %d", l)
            return related_data, related_rsids

        for rsid in rsids_from_genes:
            if not rsid in snps_by_rsid:
                logging.log(5, "related rsid %s not present in genotype data", rsid)
                continue

            related_snp = snps_by_rsid[rsid]
            freq = sum(related_snp.data)*1.0/(2*len(related_snp.data))
            if self.min_maf_filter and self.min_maf_filter > freq:
                logging.log(6, "related rsid %s below min maf: %s", rsid, freq)
                continue

            if self.max_maf_filter and self.max_maf_filter < freq:
                logging.log(6, "related rsid %s  above max maf: %s", rsid, freq)
                continue

            data = related_snp.data
            weight = weights_in_gene[rsid]
            if weight.ref_allele == related_snp.eff_allele and\
                weight.eff_allele == related_snp.ref_allele:
                logging.log(7, "related rsid %s has alleles flipped compared to model, transforming dosage", rsid)
                data = map(lambda x: 2-x, data)

            related_data.append(data)
            related_rsids.append(rsid)
        return related_rsids, related_data

    def buildMatrixOutputEntries(self, matrix, rsids_from_genes, related_rsids, snps_by_rsid):
        entries = []

        #special case: we might have a single rsid!
        if matrix.ndim == 0:
            c = str(float(matrix))
            r = rsids_from_genes[0]
            entries.append((r,r,c))
            return entries

        for i in xrange(0, len(rsids_from_genes)):
            rsid_i = rsids_from_genes[i]
            related_i = -1
            if rsid_i in related_rsids:
                related_i = related_rsids.index(rsid_i)

            for j in xrange(0, len(rsids_from_genes)):
                rsid_j = rsids_from_genes[j]

                related_j = -1
                if rsid_j in related_rsids:
                    related_j = related_rsids.index(rsid_j)

                value = "NA"
                if related_i > -1 and related_j > -1:
                    value = str(matrix[related_i][related_j])

                if i == j:
                    entries.append((rsid_i, rsid_i, value))
                else:
                    if value == "NA":
                        if rsid_i < rsid_j:
                            entries.append((rsid_i, rsid_j, value))
                    else:
                        snp_i = snps_by_rsid[rsid_i]
                        snp_j = snps_by_rsid[rsid_j]

                        if snp_i.position < snp_j.position:
                            entries.append((rsid_i, rsid_j, value))
        return entries

    def addToCorrelationFile(self, weight_db_logic, name, snps, snps_by_rsid):
        logging.info("Building correlation database for %s-%s", name, self.weight_db)
        genes = weight_db_logic.weights_by_gene.keys()
        total_genes = len(genes)
        last_reported_percent = 0
        processed = 0
        for gene in genes:
            processed += 1
            percent = int(processed*100.0 / total_genes)
            if percent == last_reported_percent+10:
                logging.info("%d percent genes processed", percent)
                last_reported_percent = percent

            entries = self.buildCorrelationEntries(name, gene, weight_db_logic, snps_by_rsid)

            if len(entries) == 0:
                logging.log(6,"Gene %s has no snps in current file", gene)
                continue

            self.addToFile(self.correlation_output, gene, entries)

    def buildCorrelationEntries(self, name, gene, weight_db_logic, snps_by_rsid):
        weights_in_gene = weight_db_logic.weights_by_gene[gene]
        rsids_from_genes = weights_in_gene.keys()

        #gather as much data as we can work on
        related_rsids, related_data = self.buildRelatedData(rsids_from_genes, snps_by_rsid, weights_in_gene)

        if len(related_rsids) == 0:
            return []

        self.updateFoundCorrelation(gene, name)

        #correlation matrix of related SNP's data
        array = numpy.array(related_data)
        cor = numpy.corrcoef(array)

        #translate into sql entries
        entries = self.buildMatrixOutputEntries(cor, rsids_from_genes, related_rsids, snps_by_rsid)
        if not len(entries):
            raise NameError("Couldn not build correlation entries for (%s,%s)" %(name,gene))
        return entries

    def updateFoundCorrelation(self, gene, name):
        found = None
        if gene in self.found_genes_for_correlation:
            found = self.found_genes_for_correlation[gene]
            logging.info("Gene %s found again for %s", gene, name)
        else:
            found = []
            self.found_genes_for_correlation[gene] = found
        found.append(name)
        

### test argument


In [12]:
import argparse


### R interface


In [5]:
import rpy2



In [8]:
from rpy2.robjects import r
import rpy2.robjects.packages as rpackages

In [10]:
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1) # select the first mirror in the list


rpy2.rinterface.NULL

In [11]:
# R package names
names_to_install = ['GBJ']

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.

if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: also installing the dependencies ‘Rcpp’, ‘mvtnorm’, ‘SKAT’, ‘BH’


  warnings.warn(x, RRuntimeWarning)
/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: trying URL 'https://cloud.r-project.org/src/contrib/Rcpp_0.12.14.tar.gz'

  warnings.warn(x, RRuntimeWarning)
/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Content type 'application/x-gzip'
  warnings.warn(x, RRuntimeWarning)
/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning:  length 3756532 bytes (3.6 MB)

  warnings.warn(x, RRuntimeWarning)
/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: =
  warnings.warn(x, RRuntimeWarning)
/Users/jerome/anaconda3/envs/me

In [13]:
importr("GBJ")

/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in loadNamespace(name) : there is no package called ‘GBJ’

  warnings.warn(x, RRuntimeWarning)
/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: In addition: 
  warnings.warn(x, RRuntimeWarning)
/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Warning messages:

  warnings.warn(x, RRuntimeWarning)
/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 1: 
  warnings.warn(x, RRuntimeWarning)
/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: In (function (pkgs, lib, repos = getOption("repos"), contriburl = contrib.url(repos,  :
  warnings.warn(x, RRuntimeWarning)
/Users/jerome/anaconda3/envs/metaxcan/lib/python2.

RRuntimeError: Error in loadNamespace(name) : there is no package called ‘GBJ’


# db

In [15]:
import sys
import os
import numpy as np
import pandas as pd
import sqlite3


# function of creating connection
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
        sys.exit(1)
    return None



In [24]:
dbname = "/Users/jerome/Projects/metaxcan/MetaXcan/v1/data/Stomach.db"

In [41]:
conn = create_connection(dbname)
cur = conn.cursor() 
gene = '"A3GALT2"'
sql = "select * from weights where gene =" + gene
tmp_query = cur.execute(sql).fetchall()

In [32]:
sql


'select * from weights where gene ="A3GALT2"'

In [42]:
tmp_query


[(u'rs644879', u'A3GALT2', 6.27340257627e-05, u'T', u'C'),
 (u'rs6671539', u'A3GALT2', -0.000263706411293, u'G', u'A'),
 (u'rs34665119', u'A3GALT2', 0.00136937184868, u'A', u'G'),
 (u'rs11802938', u'A3GALT2', 0.000224462896528, u'G', u'A'),
 (u'rs1566230', u'A3GALT2', 0.000213158781561, u'T', u'C'),
 (u'rs12040608', u'A3GALT2', 2.0053067055e-05, u'C', u'T'),
 (u'rs12040669', u'A3GALT2', 0.00213220904559, u'C', u'T'),
 (u'rs10914671', u'A3GALT2', -0.000780960140634, u'C', u'T'),
 (u'rs6670552', u'A3GALT2', -0.000323079018941, u'C', u'T'),
 (u'rs12751707', u'A3GALT2', 0.000545219903371, u'A', u'G'),
 (u'rs4077439', u'A3GALT2', -0.00177721129844, u'G', u'A'),
 (u'rs12736971', u'A3GALT2', 0.003386056752, u'A', u'G'),
 (u'rs12567237', u'A3GALT2', 0.000969534243751, u'T', u'C'),
 (u'rs28880434', u'A3GALT2', 0.000673745385822, u'C', u'T'),
 (u'rs12726791', u'A3GALT2', 0.000642417574073, u'C', u'T'),
 (u'rs12723104', u'A3GALT2', 0.00112388295962, u'T', u'C'),
 (u'rs4641276', u'A3GALT2', -0.001

In [135]:
sql = "select rsid, ref from weights where gene =" + gene

tmp_query = cur.execute(sql).fetchall()

OperationalError: no such column: ref

In [140]:
cur.execute("PRAGMA table_info(weights)").fetchall()


[(0, u'rsid', u'TEXT', 0, None, 0),
 (1, u'gene', u'TEXT', 0, None, 0),
 (2, u'weight', u'DOUBLE', 0, None, 0),
 (3, u'ref_allele', u'CHARACTER', 0, None, 0),
 (4, u'eff_allele', u'CHARACTER', 0, None, 0)]

In [45]:
str(tmp_query[0])
tmp_rsid = map(lambda x: str(x[0]), tmp_query)


In [46]:
tmp_rsid


['rs644879',
 'rs6671539',
 'rs34665119',
 'rs11802938',
 'rs1566230',
 'rs12040608',
 'rs12040669',
 'rs10914671',
 'rs6670552',
 'rs12751707',
 'rs4077439',
 'rs12736971',
 'rs12567237',
 'rs28880434',
 'rs12726791',
 'rs12723104',
 'rs4641276',
 'rs7527638',
 'rs12041371',
 'rs7523119',
 'rs10798937',
 'rs12734172',
 'rs11061',
 'rs11554674',
 'rs4653044',
 'rs12127865',
 'rs6673519',
 'rs12041636',
 'rs12727428',
 'rs11577204',
 'rs6699152',
 'rs12033491',
 'rs4357495',
 'rs10914716',
 'rs1570686',
 'rs12032426',
 'rs12022060',
 'rs12025675',
 'rs10914726',
 'rs114013172',
 'rs12737304',
 'rs1417964',
 'rs2483842',
 'rs4350172',
 'rs477438',
 'rs4652974',
 'rs507641',
 'rs472363',
 'rs508927',
 'rs549048',
 'rs771131']

In [47]:
index = pd.match(tmp_rsid, tmp_rsid)

/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pd.match() is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [50]:
tmp_array = pd.match([1,2,3,9],[2,3,5,7])


/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pd.match() is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [51]:
type(tmp_array)

numpy.ndarray

In [53]:
tmp_array[tmp_array>-1]

array([0, 1])

In [54]:
tmp_array > -1


array([False,  True,  True, False], dtype=bool)

In [55]:
tmp_rsid[tmp_array]


TypeError: only integer scalar arrays can be converted to a scalar index

In [61]:
tmp_array[tmp_array[tmp_array>-1].tolist()]






array([-1,  0])

In [62]:
tmp_array[tmp_array>-1]

array([0, 1])

In [68]:
np.array(tmp_rsid)[np.array([1,2,3])]


array(['rs6671539', 'rs34665119', 'rs11802938'],
      dtype='|S11')

In [78]:
np.array(tmp_rsid)[:3]
c = np.array([1,2,3,4,5])
c[np.array([0,2,4])] = 7,7,7
c

array([7, 2, 7, 4, 7])

In [79]:
a = np.array([1,2,3,4,5])
b = np.array([9,1,2,3,4,5,6,7,8])
indexab = pd.match(a,b)


/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pd.match() is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [80]:
indexab


array([1, 2, 3, 4, 5])

In [82]:
indexab > 0



array([ True,  True,  True,  True,  True], dtype=bool)

In [83]:
a[indexab > 0]

array([1, 2, 3, 4, 5])

In [ ]:
#joint GBJ


In [84]:
cov_matrix = np.loadtxt("/Users/jerome/Projects/metaxcan/MetaXcan/v1/data/TOP3B.cov")

In [85]:
cov_matrix.shape


(94, 94)

In [100]:
snp_rsid = pd.read_table("/Users/jerome/Projects/metaxcan/MetaXcan/v1/data/TOP3B.snplist",header =None)
snp_rsid = list(snp_rsid.loc[:,0])

In [104]:
M = len(snp_rsid) #number of snps
N = 1
gene = "TOP3B"
weights = np.zeros(shape = (M, N))
#for i in range(N):
i = 0
dbname = "/Users/jerome/Projects/metaxcan/MetaXcan/v1/data/Stomach.db"
conn = create_connection(dbname)
cur = conn.cursor()  
sql_q = 'select * from weights where gene = "' + gene + '"'
tmp_query = cur.execute(sql_q).fetchall()
rsid_in_db = map(lambda x: str(x[0]), tmp_query)
index = pd.match(rsid_in_db, snp_rsid)
indi = index[index > -1]
# extract the weight
sql_q = 'select * from weights where gene = "' + gene + '"'
tmp_query = cur.execute(sql_q).fetchall()
tmp_weights = np.array(map(lambda x: float(x[2]), tmp_query))
weights[indi,i] = tmp_weights[index > -1]

/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pd.match() is deprecated and will be removed in a future version
  del sys.path[0]


In [94]:
sql_q

'select * from weights where gene = "TOP3B"'

In [105]:
tmp_weights

array([  5.95338282e-04,   8.96832937e-06,  -5.15409039e-05,
         3.07426030e-03,   2.27576594e-04,   5.65074472e-03,
        -4.96734167e-03,   1.56079960e-03,  -1.48322517e-03,
        -2.45901074e-04,  -6.13580956e-05,  -7.19388993e-03,
         1.24588148e-03,   4.22629443e-03,   4.07837805e-04,
         3.03761490e-03,  -1.17454223e-03,  -7.91966346e-04,
        -2.02634474e-04,   5.16420029e-03,  -1.22498382e-03,
         1.34826208e-02,   3.95381306e-03,  -1.69581402e-02,
        -1.93481755e-05,  -3.15025361e-04,  -5.27540909e-04,
        -3.08823855e-03,  -8.32554991e-04,  -3.51225620e-03,
        -6.85744264e-05,  -1.44968372e-02,  -5.64451716e-03,
        -3.05647848e-03,  -1.20068994e-04,  -1.52938942e-02,
        -1.68395398e-03,  -3.48585118e-03,  -2.10373642e-03,
        -4.77514660e-03,  -1.27122491e-02,  -5.89674135e-02,
        -2.65188078e-03,  -2.78987295e-03,  -1.44375498e-02,
        -6.02239092e-03,  -7.75847601e-03,  -2.37549158e-02,
        -1.14257057e-03,

In [102]:
snp_rsid


['rs35607949',
 'rs8141374',
 'rs1005579',
 'rs9610631',
 'rs2266965',
 'rs1988315',
 'rs2266966',
 'rs5995257',
 'rs4822783',
 'rs389624',
 'rs11490038',
 'rs10427625',
 'rs79217851',
 'rs2330019',
 'rs2330018',
 'rs4145537',
 'rs2541937',
 'rs2266979',
 'rs2266977',
 'rs449597',
 'rs2283797',
 'rs9610704',
 'rs5756323',
 'rs11089937',
 'rs151680',
 'rs9620740',
 'rs5997304',
 'rs410356',
 'rs743517',
 'rs743516',
 'rs9610794',
 'rs8139376',
 'rs9610796',
 'rs2329884',
 'rs11089841',
 'rs867117',
 'rs743526',
 'rs4821629',
 'rs238775',
 'rs12158712',
 'rs9607413',
 'rs56406117',
 'rs4820285',
 'rs9610769',
 'rs8141816',
 'rs2083565',
 'rs6005623',
 'rs2027789',
 'rs239915',
 'rs239914',
 'rs9306459',
 'rs5750831',
 'rs9607401',
 'rs450129',
 'rs9607241',
 'rs56021049',
 'rs62234965',
 'rs1669124',
 'rs9625334',
 'rs5750426',
 'rs5750553',
 'rs5750552',
 'rs2266980',
 'rs5750807',
 'rs369463',
 'rs6001231',
 'rs9607473',
 'rs6001121',
 'rs882097',
 'rs9611215',
 'rs861787',
 'rs894095'

In [106]:
weights



array([[ -2.44075018e-04],
       [ -1.22498382e-03],
       [ -6.02239092e-03],
       [ -1.68395398e-03],
       [ -3.15025361e-04],
       [ -1.09079159e-04],
       [ -5.27540909e-04],
       [ -1.27122491e-02],
       [  5.95338282e-04],
       [ -5.15409039e-05],
       [ -8.32554991e-04],
       [ -1.34850193e-02],
       [ -6.85744264e-05],
       [  9.78015048e-04],
       [  4.82302711e-06],
       [ -1.05019318e-04],
       [  5.65074472e-03],
       [ -4.67824169e-03],
       [ -1.85068864e-02],
       [  1.24588148e-03],
       [ -2.39023675e-02],
       [ -1.14257057e-03],
       [ -1.04272965e-02],
       [ -1.62197066e-03],
       [ -3.05647848e-03],
       [  2.27576594e-04],
       [ -1.48322517e-03],
       [  1.56079960e-03],
       [ -2.29821392e-03],
       [ -1.69497148e-04],
       [ -2.24636899e-02],
       [ -1.93481755e-05],
       [ -2.64026795e-02],
       [ -3.48585118e-03],
       [ -1.21523565e-02],
       [ -8.04822266e-03],
       [ -8.12975278e-03],
 

In [107]:
cov_gene = weights.T * cov_matrix * weights

In [110]:
cov_gene.shape
cov_gene[0,:6]

array([  2.91712493e-08,   2.32080179e-10,   1.34355786e-08,
        -1.35915292e-09,   1.23629803e-10,   1.30368743e-08])

In [112]:
for i in range(N):
    if cov_gene[i,i] != 0:
        cov_gene[i,:] = cov_gene[i,:] / np.sqrt(cov_gene[i,i])
        cov_gene[:,i] = cov_gene[:,i] / cov_gene[i,i]

In [113]:
cov_gene[0,:6]


array([  1.00000000e+00,   1.35881564e-06,   7.86645134e-05,
        -7.95775946e-06,   7.23845142e-07,   7.63301234e-05])

### mask file

In [114]:
zscore_dict = {}
i = 0
nam = "zscore_" + str(i+1)
zscore_dict[nam] = pd.read_csv("/Users/jerome/Projects/metaxcan/MetaXcan/v1/data/Stomach.csv", header = "infer")


In [117]:
df = zscore_dict[nam]
df

,gene,gene_name,zscore,effect_size,pvalue,var_g,pred_perf_r2,pred_perf_pval,pred_perf_qval,n_snps_used,n_snps_in_cov,n_snps_in_model
0,CEACAM19,CEACAM19,7.570289,0.494302,3.723957e-14,3.791226e-02,0.5,1.000000e-10,1.000000e-10,143,162.0,163
1,EXTL3,EXTL3,NaN,-23.451963,NaN,1.264877e-05,0.5,1.000000e-10,1.000000e-10,4,4.0,4
2,CLPTM1,CLPTM1,NaN,-1.099350,NaN,5.464133e-03,0.5,1.000000e-10,1.000000e-10,21,22.0,22
3,CLU,CLU,6.973488,0.043537,3.091788e-12,3.343004e+00,0.5,1.000000e-10,1.000000e-10,21,21.0,21
4,BCAM,BCAM,NaN,2.538550,NaN,9.117730e-04,0.5,1.000000e-10,1.000000e-10,1,1.0,1
5,PPP1R37,PPP1R37,6.430535,15.715981,1.271553e-10,3.665028e-05,0.5,1.000000e-10,1.000000e-10,5,5.0,6
6,EPHA1,EPHA1,6.103094,1.449575,1.040343e-09,2.392429e-03,0.5,1.000000e-10,1.000000e-10,26,27.0,27
7,CASP2,CASP2,NaN,3.117632,NaN,4.083994e-04,0.5,1.000000e-10,1.000000e-10,17,20.0,20
8,MTCH2,MTCH2,-5.462552,-2.070170,4.693376e-08,9.155414e-04,0.5,1.000000e-10,1.000000e-10,36,37.0,38
9,CD2AP,CD2AP,4.860381,0.291114,1.171603e-06,3.277740e-02,0.5,1.000000e-10,1.000000e-10,37,37.0,37


In [118]:
df["gene"]


0         CEACAM19
1            EXTL3
2           CLPTM1
3              CLU
4             BCAM
5          PPP1R37
6            EPHA1
7            CASP2
8            MTCH2
9            CD2AP
10           SYTL2
11         FAM180B
12          PTGDR2
13          NDUFS3
14           WDR55
15           AIMP2
16            CD14
17             ZP1
18            MAFK
19           RAPSN
20           HBEGF
21            DMWD
22             MDK
23          POLR2E
24          ZNF230
25         C1QTNF4
26           MARK2
27         PCDHA10
28             PVR
29           ABCB9
           ...    
13839         BCL6
13840      FAM129C
13841        PVRL2
13842         M6PR
13843      SLC2A14
13844       CLSTN3
13845        CD163
13846         RTN2
13847           C7
13848        APOC2
13849        MFAP5
13850      PLEKHG6
13851      CD163L1
13852       TRIM40
13853       NECAP1
13854       ZNF180
13855        KLRG1
13856        VAC14
13857     ANKRD30B
13858       RIMKLB
13859    GABARAPL1
13860       

In [120]:
index = zscore_dict[nam]["gene"] == gene
index

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
13839    False
13840    False
13841    False
13842    False
13843    False
13844    False
13845    False
13846    False
13847    False
13848    False
13849    False
13850    False
13851    False
13852    False
13853    False
13854    False
13855    False
13856    False
13857    False
13858    False
13859    False
13860    False
13861    False
13862    False
13863    False
13864    False
13865    False
13866    False
13867    False
13868    False
Name: gene, Length: 13869, dtype: bool

In [121]:
sum(index)

1

In [126]:
cov_gene.shape
cov_gene[:3,:3]


array([[  1.00000000e+00,   1.35881564e-06,   7.86645134e-05],
       [  1.35881564e-06,   6.11736973e-07,   1.10037953e-06],
       [  7.86645134e-05,   1.10037953e-06,   1.90250017e-05]])

In [124]:
cov_gene[1,5]

1.0371856561492391e-10

In [125]:
cov_gene[5,1]

1.0371856561492392e-10

In [130]:
np.allclose(cov_gene,cov_gene.T)

True

In [127]:
cov_gene[8,2]

7.4587912223294393e-08

In [128]:
cov_gene[2,8]

7.4587912223294393e-08

### rpy test

In [131]:
import rpy2


In [132]:
def r_requirement():
    utils = rpackages.importr('utils')
    # select a mirror for R packages
    utils.chooseCRANmirror(ind=1) # select the first mirror in the list
    
    # R package names
    names_to_install = ['GBJ']

    # install R packages
    from rpy2.robjects.vectors import StrVector  
    if len(names_to_install) > 0:
        utils.install_packages(StrVector(names_to_install))

In [133]:
r_requirement()


/Users/jerome/anaconda3/envs/metaxcan/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: also installing the dependencies ‘Rcpp’, ‘mvtnorm’, ‘SKAT’


  warnings.warn(x, RRuntimeWarning)


In [134]:
importr('GBJ')

RRuntimeError: Error in loadNamespace(name) : there is no package called ‘GBJ’
